# 4th Programming: Shor's algorithm 

This chapter is completely concerned with Shor's algorithm: a gem, the crown jewel, of quantum computing.   
Its exponential speedup paired with the hot use case of encryption breaking is not only making computer scientists', but also politicans' breath hitch: quantum computers have the potential to break encryption protocols and intercept (previously) save data. 
To understand that use-case better, we will first have a look at the  encryption protocol that is endangered by Shor. Shor's algorithm is quite math-heavy, so we will also go through some number-theory to get a better intuition. After an in detail walk through the steps, we will show an example implemented in Qrisp. 

The algorithm can be seen as a sandwich: quantum deli between to slices of classical computing. Like many successful quantum routines, this is a hybrid algorithm, utilizing time-efficient classical steps and propose new, quantum solutions for the parts that up the complexity. 


## RSA Encryption: Really Safe Algorithm?  

Let's start with an encryption protocol, which is the reason of Shor's algorithm's relevance. A good chunk of the internet's encryption is done with RSA (named after its inventors Rivesr-Shamir-Adleman), which relies on the factorization of large numbers. When I send you a message, two prime numbers $p$ and $q$ are generated and their product $n$  is calculated. This product is further altered with the Euler-Totient-function $\phi(n) = (p-1)*(q-1)$, which calculates how many numbers smaller than $n$ are co-prime with it. Next, we are choosing an exponent $e$ that is smaller than and co-prime with $\phi(n)$. We also choose an decryption exponent $d$, such that $e*d = 1$ $mod$ $\phi(n)$ ($d$ is therefore the multiplicative inverse of $e$). We now have two different key sets: the public key $(n, e)$ and the private key $(n, d)$.  

If you want to send a message $m$ with these keys, you calculate $m^e$ $mod$ $n = c$ to encrypt your message. The recipient can now decrypt your message with $c^d$ $mod$ $n = m$.

This whole process is based on the assumption that if you take large enough numbers $p$ and $q$, the calculation for an outsider would take too long and breaking the encryption isn't feasible. This is mostly based on the factorization: When breaking the encryption and reading messages we aren't supposed to read, we need to calculate $d$ out of $n$ and $e$, which are publicly available. How is Shor's algorithm doing this? For that answer, we need to delve deeper into number theory.


## Math

At its core, Shor's algorithm is concerned with integer factorization, meaning it finds the prime actors of any given integer. 

A mathematical property is helping us with that: If we have two Numbers $A$ and $B$ that are co-prime, the equation $A*A*A*A*A*... = A^p = C*B+1$ holds up. Meaning that if we raise $A$ to an unknown power, this shares a factor with another multiply of $B$ plus 1, or $A^p -1 = C*B = (A^{\frac{p}{2}}+1)* (A^{\frac{p}{2}}-1)$. This form looks like our initial problem again, where we search for two factors $C*B = D*E$ with $D= A^{\frac{p}{2}} +1$ and $E= A^{\frac{p}{2}} -1$. These numbers now share factors. So with the rule $A^p = C*B+1$, we could transform two numbers that don't share factors into four who do. Notice how the $p$ we need is not specified, so we will look for this instead in Shor's algorithm.   
This equation is derived from Euler's modular theorem, stating that if $x$ and $n$ are co-prime, $x^{\phi(n)} = 1$ $mod$ $n$, which means it's equal to a multiple of $n$ plus 1. In this case, $x=A$ and $n=B$ in our above equation. 
Because we use modular arithmetic, this problem is periodic. $A^0 = 1$ $mod$ $N$ is a trivial solution, and we are now loking for the period $p$ after which we again get $A^x = 1$ $mod$ $N$. 

The second quality we will use states that if $r^x = m_1*N+ d$, then $r^{x+p} = m_2 *N + d$, meaning that for any multiple of $p$, the remainder stays the same (with another multiple of $N$). This repeating property will help us identify $p$ later on. 

Another (classical) step in Shor's algorithm is Euclid's algorithm to find the greatest common divisor of two numbers. Utilizing this, we don't need to find a number that is a direct factor of $N$, but they can also share factors. In this routine, you are given two numbers and subtract mutltiples from them the bigger one untill you are left with 0. The last number you subtracted the the greatest common divisor. This has a runtime of $O((log(a*b))^3)$.


## The Algorithm 

How can you determine the factors of a ridiculously large number $N$? First, we might as well pick any random number that is smaller than our given one as an initial guess. This will most likely not be or share a factor with $N$, but we can check using the Euclidean algorithm. If we find that $gcd(N, r)=1$, we now transform $r \rightarrow  r^{\frac{p}{2}} \pm 1$ (as per the equation we introduced above), which now might be multiples of our sought out factors (but with Euclid, that also isn't a problem). However, other problems are posed:    
$A^{\frac{p}{2}} +1$ and $A^{\frac{p}{2}} -1$ might be a multiple of $N$, so the common divisor we would find is $N$ itself and we gain nothing.
If $p$ is an odd number, how can we find its half? How can we even find $p$? Finding the power to raise our guess to could take a long time on a classical computer, and this is where we enter the quantumness with interference.  

As we have explained some time ago in the physics chapter, interference happens if two or more waveforms collide and the waves either add up or cancel each other out into a new pattern. In this case, we want it to cancel out all wrong solutions, so that we are only left with the state that has the correct $p$. How do we do this? We start with our guess $r$ and a random $x$. We raise $r^x$ and put all possible powers into a superposition $\ket{\psi_1} = \ket{x, r^x} + \ket{x+1, r^{x+1}}+ \ket{x+2, r^{x+2}}...$. We now want to calculate the remainder $d$ from how much $r^x$ is bigger than $m*N$ to gain $\ket{\psi_2}= \ket{x, d_x} + \ket{x+1, d_{x+1}}+ \ket{x+2, d_{x+2}}...$. The next step is to cancel out all states that don't carry $d= 1$ with interference. How can we implement this? 

After this step, all $x$ in the remaining states are spaced out with period $p$, meaning that we have a frequency of $f= \frac{1}{p}$. Frequency? This word triggers the entrance of another subroutine we have learned some chapters ago: Quantum Fourier Transformation. This step is central in Quantum Phase Estimation, which is used as a sub-algorithm here. As the name suggests, we estimate a phase, here the period after which $d=1$ get repeated.  
With QFT, we calculated the frequency and can invert now to get $p$. Great! Now there's nothing left to do. Well, except check is $p$ is even (and repeat the whole process if it's not), calculate $r^{\frac{p}{2}} \pm 1$ and maybe apply Euclid's algorithm again. But now, you can read Trump's e-mails (if you even want that)! 

Shor's algorithm works in polynomial time, instead of exponential with the classical method, yielding an exponential advantage.  


## Implementation with Qrisp 

The main components of Shor's algorithm are already implemented in Qrisp and you only need to plug in your values. 

As per pre-Qrisp, the only numbers that were factorized with Shor's algorithm where 15 and 21. While you might not break the white house's encryption with that, Qrisp offers you the chance to try bigger numbers. 

If the explanation behind all the doohickeys wasn't really your thing and your're just here to factor, all you need is: 


In [ ]:
from qrisp.shor import shors_alg
print(shors_alg(35))


to come to your solution. If you are actually interested in the works behind the magic, we will explain how we implemented that. 

### Behind the Scenes 

In this section, we will take a look at how we implemented Shor's algorithm so you don't have to. 

First, we check if the given number is even. 
Next, we search for possible $r$ that are co-prime with $N$


Montgomery multiplication is often used to reduce the needed steps for (modular) mutliplication on a computer.
To optimize computing power, we transform into Montgomery space beforehand. Therefore, we check for a $r$ that requires few qubits to reduce to the Montgomery space.  (TODO: explain this deeper?)  
Next, we check if our chosen $r$ might actually be a divisor of $N$ already. If it isn't, we continue with the true quantum magic. We apply modulo operations and put them into an equal superposition using a Hadamard gate to get the state $\ket{\psi_2}= \ket{x, d_x} + \ket{x+1, d_{x+1}}+ \ket{x+2, d_{x+2}}...$. We then find the phase with remainder $d=1$ using IQFT. If our solution is now even, we can transform it to $r^{\frac{p}{2}}+1$, calculate the $cgd(r^{\frac{p}{2}}+1, N)$ and we're done!


### Adder this, Adder that 

In the above command, we only specified wich number to factor, but you can also further tweak your adder. 
Why do we even need an adder, if we didn't do any addition above? You see, we used moduar exponentiation $a^s$ $mod$ $N$, which is based on multiplication, which is based on modular addition. Since we break down everything to addition, we need it quite  alot, so it makes sense to spare some thoughts to which adder we are using.   
In Qrisp, you can even define your own adder and use it, but if you are not that motivated, you can also pick and choose from one of our pre-implemented ones. The main difference of adders in their relation between ancilla qubits and T-depth. 

### Cause two can keep a secret if one of them is encrypted 

In Qrisp, you can also use crytpography tools to directly en- and decrypt RSA. These functions are implemented using Shor's algorithm. Let's say you suspect your friends Alice and Bob talking behind you back. 


In [ ]:
from qrisp.shor import rsa_encrypt_string
rsa_encrypt_string(p = 5, q = 13, e = 7, message = "brunch at 12!") # this part is done by Alice 

You intercept Alice's encrypted message to Bob: 
0110001010010100001100000110001001101110100101000001000001101100011110010110000000010101100000111100100000100001

In [ ]:
intercepted_message = '0110001010010100001100000110001001101110100101000001000001101100011110010110000000010101100000111100100000100001'
from qrisp.shor import rsa_decrypt_string
print(rsa_decrypt_string(e = 7, N = 65, ciphertext = intercepted_message))


Your friends are hanging out without you! Time for a confrontation. 


## Summary 

It can barely be overstated how much Shor's algorithm has done for the quantum computing community. Much funding, from private investors and governments alike, have been motivated by the thought that if one algorithm promises exponential advantage, there must be another. Furthermore, the threat of breaking previously safe encryption, brought up many concerns for safety and the will to find new security measures in the same world that promised to break the old ones.

So why is it that we have this knowledge, but your steamy text messages are still safe and sound? Once again, we do not have a performant enough physical device to run this algorithm. Between error-proneness and number of qubits, the tainted relationship between physical and logical qubits, we cannot utilize enough qubits to factor the big and interesting numbers. 

+ Shor's algorithm offers exponential advantage and can be used to break RSA encryption 
1. classical part:  transform $r \rightarrow  r^{\frac{p}{2}} \pm 1$ from guess $r$
2.  quantum part:  raise $r^x$ and put all possible powers into a superposition $\ket{\psi_1} = \ket{x, r^x} + \ket{x+1, r^{x+1}}+ \ket{x+2, r^{x+2}}...$, calculate remainder, cancel all states out with interference except for $d=1$. Use QPE to calculate period $p$ 
3. classical part: check if $p$ is even and calculate   $r^{\frac{p}{2}} \pm 1$ 
+ In Qrisp, just use `shors_alg(N)`